Skew-T Radiosonde plot

$ source activate py36skewT

$ jupyter notebook




In [79]:
import sys
sys.path.append('/Volumes/SANDISK128/Applications/pyMeteo/pymeteo/')
sys.path.append('/Volumes/SANDISK128/Documents/Research_UiO_50/research_Haukeli')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')

import numpy as np
import netCDF4
import pandas as pd

import thermo
from pymeteo.constants import *
from datetime import date
import datetime

import calc_date as cd
import createFolder as cF
import plot_skewT as skewT

In [80]:
savefig = 0

stn = 'Stavanger'

year = '2016'
# Nov 2016
month = '11'
t = np.arange(11,19)
t = np.append(t,[28,29])

# Dec 2016
#month = '12'
#t = np.arange(21,28)

#year = '2017'
# Jan 2017
#month = '01'
#t = np.arange(2,14)
#t = np.append(t,[28,29])

# Feb 2017
#month = '02'
#t = np.arange(1,5)

hh = '00'

if savefig == 1:
    fig_file = '../../Figures/Sounding/%s/%s%s/' %(stn,year,month)
    cF.createFolder(fig_file)

In [81]:
for day in np.arange(11,12):
    ###### OPEN files ########
    nc_file = '../../Data/MEPS/%s/%s%s/hybrid65_EM0_%s/%s%s%s_00.nc' %(stn,year,month,hh,year,month,day)
    
    fn = netCDF4.Dataset(nc_file)

In [82]:
fn

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(67), hybrid(65), ensemble_member(1)
    variables(dimensions): float64 time(time), float64 hybrid(hybrid), float64 air_temperature_ml(time,hybrid), float64 specific_humidity_ml(time,hybrid), float64 x_wind_ml(time,hybrid), float64 y_wind_ml(time,hybrid), float32 atmosphere_cloud_condensed_water_content_ml(time,hybrid), float32 atmosphere_cloud_ice_content_ml(time,hybrid), float32 pressure_departure(time,hybrid), float64 pressure_ml(time,hybrid), float64 layer_thickness(time,hybrid), float64 geop_layer_thickness(time,hybrid)
    groups: 

In [85]:
# Read in time to know initialisation time (plotting)
time = fn.variables['time'][:]
# time = fv.fill_nan(time[:])
ini_day = datetime.datetime.utcfromtimestamp(time[int(hh)]).day   # day of initialisation
hour    = datetime.datetime.utcfromtimestamp(time[int(hh)]).hour  # hour of initialisation
calday, calmon = cd.get_dayname(year,month,ini_day)

# Read in the variable names which are needed for plotting the Radiosonde
T = fn.variables['air_temperature_ml'][:,:]
p = fn.variables['pressure_ml'][:,:]
u = fn.variables['x_wind_ml'][:,:]
v = fn.variables['y_wind_ml'][:,:]
qv = fn.variables['specific_humidity_ml'][:,:]

# Layer thickness to calculate height
thickness_ml = fn.variables['layer_thickness']
h_above = []
h_above.append(np.zeros(thickness_ml[:,:].shape[0]))
th_arr = np.nansum([np.asarray(h_above)[:,0], thickness_ml[:,0]], axis = 0)
h_above.append(th_arr)
for i in range(2,thickness_ml[:,:].shape[1]):
    th_arr = np.nansum([np.array(h_above)[i-1], thickness_ml[:,i-1]],axis=0)
    h_above.append(th_arr)
height_ml = np.transpose(h_above)

# substitue missing values with nan
#T = fv.fill_nan(T[:])
#p = fv.fill_nan(p[:])
#u = fv.fill_nan(u[:])
#v = fv.fill_nan(v[:])
#qv = fv.fill_nan(qv[:])

# calculate dew point with thermo.py
Td = thermo.Td(p, qv)

# get the temperature in °C
T  = T - T00
Td = T - T00

# plot vertical temp profile
if hour < 10:
    hour = '0%s' %hour

title = '%s, %s %s %s   %s$\,$UTC' %(calday,ini_day,calmon,year,hour)
skewT.plot_skewT(T, Td, height_ml, p, u, v, title)
#skewT.plot_skewT_EM(T, Td, height_ml, p, u, v, int(hh), title)

if savefig == 1:
    filename = fig_file+'%s%s%s_%s.png' %(year,month,ini_day,hh)
    plt.savefig(filename,orientation = 'portrait', papertype = 'a4')#, dpi=300,bbox_inches=0)
else:
    plt.show()
plt.close()    
    




ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()